# 09. Работа с внешними API

## Вступление

В этом уроке используются библиотеки apiclient и oauth2client. Вы можете скачайте этот файл себе на компьютер, зарегистрироваться в Google и установить библиотеки локально, а можете воспользоваться серверной версией и поработать под уже созданным аккаунтом.

## 1. Авторизация

Сегодня много информации в интернете можно получить бесплатно и в удобном для дальнейшего использования виде (JSON или XML). 
Курсы валюты, прогноз погоды, информацию о пробках или географических объектах.

Все взаимодействие с информационным сервисами происходит через их API - открытые для сторонних пользователей функции, с доступной документацией и даже примерами кода на разных языках.

Иногда бесплатно доступны только базовые функции или стоит ограничение на количество запросов. 

Чтобы отслеживать это количество запросов (а еще для сбора маркетинговой статистики и рассылки рекламы) даже бесплатные сервисы требуют регистрацию и выдают информацию только после авторизации.

На сегодняшний день можно выделить три основных способа авторизации.
1. Сервис при регистрации выдает ключ (ApiKey, APPID, Application Key), который нужно передавать в каждом запросе. Это удобно, достаточно одного служебного параметра чтобы получить данные.
2. Устаревший способ - сервис выдает логин и пароль. При первом обращении к сервису нужно авторизоваться при помощи специальной функции, предоставляемой сервисом, в ответ получить уникальный идентификатор сессии и передавать его в каждом запросе (как ключ из предыдущего пункта). У этого идентификатора есть время жизни и через определенный срок он перестанет действовать. Тогда нужно снова авторизоваться. Я не смог найти ни одного бесплатного сервиса, который бы использовал этот способ. Но нужно знать, что такое возможно.
3. Наиболее продвинутые сервисы предоставляют специально разработанные Python-библиотеки для доступа к своим сервисам. В них при первом обращении нужно авторизоваться, используя выданный ключ, а ответ библиотека вернет Python-объект, с помощью которого можно осуществлять все дальнейшие действия. 
Ниже мы разберем примеры работы по способам 1 и 3.


## 2. Получаем прогноз погоды

Сайт https://openweathermap.org/ предоставляет множество погодных сервисов. Чтобы пользоваться ими нужно пройти регистрацию на сайте и получить ключ (ApiKey). Я уже прошел регистрацию и получил ключ, именно им мы и будет пользоваться.

Если вы захотите использовать этот сайт для вашей работы, вам следует зарегистрироваться на сайте и получить свой ключ.

Авторизация проходит по способу 1.

Прогноз погоды можно получить по коду города или географическим координатам. А код города можно получить выполнив запрос.

Адрес запросе http://api.openweathermap.org/data/2.5/find

Параметры:

- q - Название города на английском
- type - тип сопоставления. Будем использовать like - название города должно включать в себя текст, указанный в запросе.
- units - в каких градусах указывать температуру. В данном случае это не имеет значение, но в запросах погоды будет использовать metric - градусы Цельция.
- APPID - код для авторизации. В нашем примере используется код c6dfe45cf458e6208c3a8c3d63269451

Наиболее простой вариант получения ответа - просто просто отправить запрос при помощи библиотеки requests, соединив все параметры в одну адресную строку, и получить ответ


In [ ]:
import requests
r = requests.get("http://api.openweathermap.org/data/2.5/find?q=Moscow&type=like&APPID=c6dfe45cf458e6208c3a8c3d63269451")
r.json()

Это не самый наглядный вариант. Параметры замешаны в строке и их трудно воспринимать. Перепишем запрос

In [ ]:
import requests
s_city = "Moscow"
appid = 'c6dfe45cf458e6208c3a8c3d63269451'
r = requests.get("http://api.openweathermap.org/data/2.5/find",
                 params={"q": s_city, "type": "like", "units": "metric", "APPID": appid})
r.json()

Так гораздо нагляднее и понятнее.

Проанализировав json-ответ пишем код, который будет выводить список найденных городов.

In [ ]:
import requests
s_city = "Moscow"
appid = 'c6dfe45cf458e6208c3a8c3d63269451'
r = requests.get("http://api.openweathermap.org/data/2.5/find",
                 params={"q": s_city, "type": "like", "units": "metric", "APPID": appid})
for s in r.json()['list'] :
    print(s['id'], s['name'], s['sys']['country'])

Этот скрипт выводит ID города, название и страну.

Возьмет код Москвы (той что в России) - 524901. 

Запросим погоду в Москве

In [1]:
import requests
appid = 'c6dfe45cf458e6208c3a8c3d63269451'
city_id = 524901 # Код Москвы
res = requests.get("http://api.openweathermap.org/data/2.5/weather", # Формируем запрос
                 params={'id': city_id, 'units': 'metric', 'lang': 'ru', 'APPID': appid})
res.json()

{'coord': {'lon': 37.62, 'lat': 55.75},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'ясно',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 15.36,
  'pressure': 1006,
  'humidity': 77,
  'temp_min': 15,
  'temp_max': 16.11},
 'visibility': 10000,
 'wind': {'speed': 3, 'deg': 210, 'gust': 8},
 'clouds': {'all': 0},
 'dt': 1570042623,
 'sys': {'type': 1,
  'id': 9027,
  'message': 0.0145,
  'country': 'RU',
  'sunrise': 1569987190,
  'sunset': 1570028683},
 'timezone': 10800,
 'id': 524901,
 'name': 'Moscow',
 'cod': 200}

Проанализировав json выберем из него самые важные параметры и допишем код, чтобы вывод данных был более наглядным

In [2]:
import requests
from IPython.display import Image # Библиотека для отображения картинок
from IPython.core.display import HTML # Библиотека для отображения HTML
appid = 'c6dfe45cf458e6208c3a8c3d63269451'
city_id = 524901 # Код Москвы
res = requests.get("http://api.openweathermap.org/data/2.5/weather", # Формируем запрос
                 params={'id': city_id, 'units': 'metric', 'lang': 'ru', 'APPID': appid})
data = res.json() # Получает ответ в JSON
print("Погода:", data['weather'][0]['description'])
print("Температура в данное время:", data['main']['temp'])
print("Минимальная температура сегодня:", data['main']['temp_min'])
print("Максимальная температура сегодня:", data['main']['temp_max'])
# Выводим иконку текущей погоды. 
display(Image(url= 'http://openweathermap.org/img/wn/' + data['weather'][0]['icon'] + '.png'))

Погода: ясно
Температура в данное время: 15.36
Минимальная температура сегодня: 15
Максимальная температура сегодня: 16.11


В этом сервисе есть еще одна функция - прогноз погоды на 5 дней с шагом в 3 часа.
Нужно изменить только адрес запроса - http://api.openweathermap.org/data/2.5/forecast - остальные параметры остаются прежними.

In [3]:
import requests
from IPython.display import Image # Библиотека для отображения картинок
from IPython.core.display import HTML # Библиотека для отображения HTML
appid = 'c6dfe45cf458e6208c3a8c3d63269451'

city_id = 524901 # Код Москвы


res = requests.get("http://api.openweathermap.org/data/2.5/forecast",
                           params={'id': city_id, 'units': 'metric', 'lang': 'ru', 'APPID': appid})
data = res.json()
data 

{'cod': '200',
 'message': 0.0091,
 'cnt': 40,
 'list': [{'dt': 1570050000,
   'main': {'temp': 13.86,
    'temp_min': 11.87,
    'temp_max': 13.86,
    'pressure': 1007.76,
    'sea_level': 1007.76,
    'grnd_level': 985.71,
    'humidity': 91,
    'temp_kf': 1.99},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'легкий дождь',
     'icon': '10n'}],
   'clouds': {'all': 99},
   'wind': {'speed': 3.19, 'deg': 247.675},
   'rain': {'3h': 0.812},
   'sys': {'pod': 'n'},
   'dt_txt': '2019-10-02 21:00:00'},
  {'dt': 1570060800,
   'main': {'temp': 12.79,
    'temp_min': 11.3,
    'temp_max': 12.79,
    'pressure': 1009.29,
    'sea_level': 1009.29,
    'grnd_level': 986.97,
    'humidity': 95,
    'temp_kf': 1.49},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'легкий дождь',
     'icon': '10n'}],
   'clouds': {'all': 100},
   'wind': {'speed': 1.67, 'deg': 192.38},
   'rain': {'3h': 1.376},
   'sys': {'pod': 'n'},
   'dt_txt': '2019-10-03 00:00:

##### Задание для самостоятельного выполнения:

- На примере предыдущего скрипта доработайте этот код, чтобы он выводит прогноз в человекочитаемом виде (дата-время, температура, скорость ветра и русскоязычное описание погоды)

## 3. Работа с Google Sheets

Поработаем над более сложной задачей. Научимся создавать Google-таблицы, вставлять в них данные, красить ячейки, выделять заголовки, и читать данные из уже созданных таблиц.

Для этого придется зарегистрироваться на Google, настроить проект и локально, на ваш компьютер, установить необходимые библиотеки.

Вся работа будет вестись только через эти библиотеки, никаких запросов напрямую, как в предыдущем примере.

Официальная документация на английском языке находится по адресу https://developers.google.com/sheets/

## 4. Регистрация в сервисах Google и установка библиотек

Сначала нужно зарегистрироваться на gmail.com (это вы можете сделать самостоятельно).
Потом нужно создать проект (так Google предоставляет доступ к своим сервисам).

Это долгий и нудный процесс, который позволяет понять, почему интерфейсы разработчиков Google называют самыми неудобными в мире (некоторые считают, что социальная сеть Google+ не взлетела именно по этой причине).

Для этого зайдите на страницу
https://console.developers.google.com/cloud-resource-manager и нажать “Создать проект”


<img src="img/Task09_01.jpg">

Введите имя проекта и нажмите “Создать”

<img src="img/Task09_02.jpg">

В обновленном списке проектов зайдите в меню “Права доступа”

<img src="img/Task09_03.jpg">

В открывшемся окне нажмите “Добавить”, внесите свой email с домена gmail.com и выберите группу “Проект” - “Владелец”

<img src="img/Task09_04.jpg">

Сохраните изменения.

Может показаться странным, что вы создали проект, но вынуждены сами себе выдавать права. И это на самом деле странно, но именно такой путь пришлось пройти на момент написания этого курса, чтобы все начало работать как надо.


Снова зайдите на страницу https://console.developers.google.com/cloud-resource-manager

Выберите на своем проекте меню “Настройки” 


<img src="img/Task09_05.jpg">

В открывшемся окне выберите “Сервисные аккаунты”, а затем “Создать сервисный аккаунт”

<img src="img/Task09_06.jpg">

Введите название аккаунта и нажмите “Создать”

<img src="img/Task09_07.jpg">

Выберите роль “Владелец” и нажмите “Продолжить” 

<img src="img/Task09_08.jpg">

В появившемся окне нажмите "Создать ключ"

<img src="img/Task09_09.jpg">

Выберите тип ключа “json” и нажмите “Создать”

<img src="img/Task09_10.jpg">

Будет создан и сразу же скачан файл с ключами. Сохраните его, именно благодаря ему мы сможем получать доступ к сервисам Гугл.

Нажмите на кнопку с тремя горизонтальными штрихами, слева от надписи “Google APIs”, выберите пункт “API и сервисы”, а в нем подпункт “Панель управления”.

<img src="img/Task09_11.jpg">

В открывшемся окне нажмите “Включить API и сервисы”

<img src="img/Task09_12.jpg">

Введите в строку поиска “google drive” и кликните на сервисе “Google Drive API”

<img src="img/Task09_13.jpg">

Нажмите “Включить”

<img src="img/Task09_14.jpg">

Сайт уведомит вас, что API включено и предупредит, что нужно создать учетные данные. Игнорируйте это предупреждение (мы уже создали сервисный аккаунт, но по непонятной причине Гугл забыл об этом).

<img src="img/Task09_15.jpg">

Снова заходите в панель управления

<img src="img/Task09_16.jpg">

В открывшемся окне нажмите “Включить API и сервисы”

<img src="img/Task09_12.jpg">

Введите в строку поиска “sheet” и кликните на сервисе “Google Sheets API”

<img src="img/Task09_17.jpg">

Убедитесь, что это API подключено. Оно должно включиться автоматически, при подключении Google Drive API. Если оно подключено, вы увидите кнопку “Управление API”, если нет - кнопку “Включить”. Включите его, при необходимости.

<img src="img/Task09_17-1.jpg">

В последний раз зайдите на страницу https://console.developers.google.com/cloud-resource-manager

Выберите на своем проекте меню “Настройки” 

<img src="img/Task09_17-2.jpg">

В открывшемся окне выберите “Сервисные аккаунты”, а затем скопируйте и сохраните email сервисного аккаунта. Он пригодится вам, чтобы выдавать доступ к таблицам.

<img src="img/Task09_17-3.jpg">

Теперь переходим к установке библиотек. Выполните команду 

In [1]:
! pip3 install --upgrade google-api-python-client

  Created wheel for google-api-python-client: filename=google_api_python_client-1.7.11-cp36-none-any.whl size=56535 sha256=0b618f9cb220f60f894c9496aaf97f9fa5f04c9f3886fc10cdf6c7a6e32ff3d0
  Stored in directory: C:\Users\ovk\AppData\Local\pip\Cache\wheels\b8\f6\f5\b7bef1a5bc0e07ca4aa54c596b0b574c5afc07a9fddccf08f8
Successfully built google-api-python-client
  Found existing installation: google-api-python-client 1.7.10
    Uninstalling google-api-python-client-1.7.10:


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'c:\\program files\\python36\\lib\\site-packages\\apiclient\\__init__.py'
Consider using the `--user` option or check the permissions.



а затем 

In [ ]:
! pip3 install oauth2client 

Возможно, что при запуске второй команды вы получите сообщение, что библиотека oauth2client уже установлена.   

Зайдите на страницу 
https://raw.githubusercontent.com/gsuitedevs/python-samples/master/sheets/quickstart/quickstart.py

Нажмите правую кнопку мышки и выберите “Сохранить как”

<img src="img/Task09_18.jpg">

Сохраните файл под именем quickstart.py

и запустите его командой 

python quickstart.py

Откроется новая страница в браузере (возможно, он скажет, что страница небезопасная, но смело идите вперед) и вам надо будет принять условия. 

На этом наш путь завершен.

## 5. Заполнение и форматирование таблицы

Создадим первую таблицу

In [2]:
# Подключаем библиотеки
import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials

CREDENTIALS_FILE = 'seraphic-effect-248407-7ac2c44ec709.json'  # Имя файла с закрытым ключом, вы должны подставить свое

# Читаем ключи из файла
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets',
                                                                                  'https://www.googleapis.com/auth/drive'])

httpAuth = credentials.authorize(httplib2.Http()) # Авторизуемся в системе
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API 

spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Первый тестовый документ', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Лист номер один',
                               'gridProperties': {'rowCount': 100, 'columnCount': 15}}}]
}).execute()
spreadsheetId = spreadsheet['spreadsheetId']
print('https://docs.google.com/spreadsheets/d/' + spreadsheetId)

https://docs.google.com/spreadsheets/d/1TP9cyW0juX1r5iaT9xnLUMNKCynGkT5YDRuypd3CYRU


Если все прошло без ошибок - на экран будет выведена ссылка на таблицу. 

В этой ссылки использован идентификатор файла, мы сохраняем его в переменной spreadsheetId и будем использовать в дальнейшем. 

Переходите по ней. Гугл сообщит вам, что у вас нет доступа


<img src="img/Task09_19.jpg">

Не запрашивайте разрешение! Вам придет уведомление, что невозможно доставить письмо с запросом на адрес, который сам Гугл назначил системному аккаунту. А изменить этот адрес нельзя. Возможно, это не работает только в бесплатном режиме.

Но мы можем выдать себе доступ через Google Drive. Вам нужно заменить адрес my_test_address@gmail.com на свой.

In [3]:
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) # Выбираем работу с Google Drive и 3 версию API
access = driveService.permissions().create(
    fileId = spreadsheetId,
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'my_test_address@gmail.com'},  # Открываем доступ на редактирование
    fields = 'id'
).execute()


Теперь у вас есть доступ, не закрывайте таблицу, мы будем управлять ею и сразу же смотреть на изменения.

У каждого документа есть свой код - spreadsheetId - именно от отображается в адресной строке, когда мы открываем таблицу в браузере (в URL-е страницы с открытой таблицей он находится между "https://docs.google.com/spreadsheets/d/" и "/edit#gid=0").

Мы сохранили его в переменной spreadsheetId и дальше будем с ним работать.

<img src="img/Task09_20.jpg">

Сначала немного теории. 

В каждом файле (spreadsheet) находятся листы-вкладки (sheet). 

Каждый sheet имеет свой числовой код (sheetId). У первого созданного в документе листа этот код равен 0. Остальные листы имеют сильно отличные от нуля id (т.е. они не нумеруются подряд).

Убедимся в этом

In [3]:
# Добавление листа
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": [
    {
      "addSheet": {
        "properties": {
          "title": "Еще один лист",
          "gridProperties": {
            "rowCount": 20,
            "columnCount": 12
          }
        }
      }
    }
  ]
}).execute()

In [4]:
# Получаем список листов, их ID и название
spreadsheet = service.spreadsheets().get(spreadsheetId = spreadsheetId).execute()
sheetList = spreadsheet.get('sheets')
for sheet in sheetList:
    print('Title = ', sheet['properties']['title'], 'ID = ', sheet['properties']['sheetId'])
    
sheetId = sheetList[0]['properties']['sheetId']

print('Мы будем использовать лист с ID = ', sheetId)

Title =  Лист номер один ID =  0
Мы будем использовать лист с ID =  0


В самом деле, первый лист имеет id равный нулю, а второй пронумерован иначе.

Еще один вопрос: как указывать диапазоны ячеек.
Видимо, таблицы Гугл разрабатывали разные команды, под руководством разных менеджеров и при помощи разных архитекторов.
Потому, что координаты ячеек задаются двумя разными способами.

Вариант 1: в формате текста "Лист номер один!B2:D5", т.е. имя листа, после него восклицательный знак, после - левая верхняя ячейка в формате “буква (колонка) + цифра (строка)” + правая нижняя ячейка в таком же формате.

Вариант 2: в json-формате, с указанием ID листа и координат левой верхней и правой нижней ячеек в числовом виде (номер строки и номер столбца)

    'range':
    {
    'sheetId': sheetId, # ID листа
    'startRowIndex': 1, # Со строки номер startRowIndex 
    'endRowIndex': 5,# по endRowIndex - 1 (endRowIndex не входит!)
    'startColumnIndex': 0, # Со столбца номер startColumnIndex 
    'endColumnIndex': 1 # по endColumnIndex - 1
    },

Разные функции используют разные форматы.

Теперь можно заполнить ячейки данными, нарисовать рамку и выделить заголовки.

In [5]:
results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, body = {
    "valueInputOption": "USER_ENTERED", # Данные воспринимаются, как вводимые пользователем (считается значение формул)
    "data": [
        {"range": "Лист номер один!B2:D5",
         "majorDimension": "ROWS",     # Сначала заполнять строки, затем столбцы
         "values": [["Ячейка B2", "Ячейка C2", "Ячейка D2"], # Заполняем первую строку
                    ['25', "=6*6", "=sin(3,14/2)"]  # Заполняем вторую строку
                   ]}
    ]
}).execute()

Заполняем несколько ячеек данными. Т.к. указан параметр USER_ENTERED, таблица воспринимает эти данные так, как восприняла бы ввод руками пользователя - преобразует числовые значения в числа, а значения, начинающиеся со знака “равно” в формулы. 

Посмотрите в вашу таблицу, она заполнилась данными

<img src="img/Task09_21.jpg">

Зададим ширину колонок. Функция batchUpdate может принимать несколько команд сразу, так что мы одним запросом установим ширину трех групп колонок, а одной из групп две колонки, а двух других по одной.

In [6]:
results = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheetId, body = {
  "requests": [

    # Задать ширину столбца A: 20 пикселей
    {
      "updateDimensionProperties": {
        "range": {
          "sheetId": sheetId,
          "dimension": "COLUMNS",  # Задаем ширину колонки
          "startIndex": 0,         # Нумерация начинается с нуля
          "endIndex": 1            # Со столбца номер startIndex по endIndex - 1 (endIndex не входит!)
        },
        "properties": {
          "pixelSize": 20     # Ширина в пикселях
        },
        "fields": "pixelSize"  # Указываем, что нужно использовать параметр pixelSize  
      }
    },

    # Задать ширину столбцов B и C: 150 пикселей
    {
      "updateDimensionProperties": {
        "range": {
          "sheetId": sheetId,
          "dimension": "COLUMNS",
          "startIndex": 1,
          "endIndex": 3
        },
        "properties": {
          "pixelSize": 150
        },
        "fields": "pixelSize"
      }
    },

    # Задать ширину столбца D: 200 пикселей
    {
      "updateDimensionProperties": {
        "range": {
          "sheetId": sheetId,
          "dimension": "COLUMNS",
          "startIndex": 3,
          "endIndex": 4
        },
        "properties": {
          "pixelSize": 200
        },
        "fields": "pixelSize"
      }
    }
  ]
}).execute()

Посмотрите на таблицу, ширины колонок изменились.

<img src="img/Task09_22.jpg">

Нарисуем рамку вокруг таблицы

In [7]:
# Рисуем рамку
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = {
        "requests": [
            {'updateBorders': {'range': {'sheetId': sheetId,
                             'startRowIndex': 1,
                             'endRowIndex': 3,
                             'startColumnIndex': 1,
                             'endColumnIndex': 4},
                   'bottom': {  # Задаем стиль для верхней границы
                              'style': 'SOLID', # Сплошная линия
                              'width': 1,       # Шириной 1 пиксель
                              'color': {'red': 0, 'green': 0, 'blue': 0, 'alpha': 1}}, # Черный цвет
                   'top': {     # Задаем стиль для нижней границы
                              'style': 'SOLID',
                              'width': 1,
                              'color': {'red': 0, 'green': 0, 'blue': 0, 'alpha': 1}},
                   'left': {    # Задаем стиль для левой границы
                              'style': 'SOLID',
                              'width': 1,
                              'color': {'red': 0, 'green': 0, 'blue': 0, 'alpha': 1}},
                   'right': {    # Задаем стиль для правой границы
                              'style': 'SOLID',
                              'width': 1,
                              'color': {'red': 0, 'green': 0, 'blue': 0, 'alpha': 1}},
                   'innerHorizontal': {   # Задаем стиль для внутренних горизонтальных линий
                              'style': 'SOLID',
                              'width': 1,
                              'color': {'red': 0, 'green': 0, 'blue': 0, 'alpha': 1}},
                   'innerVertical': {     # Задаем стиль для внутренних вертикальных линий
                              'style': 'SOLID',
                              'width': 1,
                              'color': {'red': 0, 'green': 0, 'blue': 0, 'alpha': 1}}
                              
                              }}
        ]
    }).execute()

<img src="img/Task09_23.jpg">

Объединим ячейки над таблицей и впишем в них заголовок

In [8]:
# Объединяем ячейки A2:D1
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = {
        "requests": [
            {'mergeCells': {'range': {'sheetId': sheetId,
                          'startRowIndex': 0,
                          'endRowIndex': 1,
                          'startColumnIndex': 1,
                          'endColumnIndex': 4},
                'mergeType': 'MERGE_ALL'}}
        ]
    }).execute()
# Добавляем заголовок таблицы
results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, body = {
    "valueInputOption": "USER_ENTERED", # Данные воспринимаются, как вводимые пользователем (считается значение формул)
    "data": [
        {"range": "Лист номер один!B1",
         "majorDimension": "ROWS",     # Сначала заполнять строки, затем столбцы
         "values": [["Заголовок таблицы" ] 
                   ]}
    ]
}).execute()

<img src="img/Task09_25.jpg">

Установим формат у ячеек заголовка таблицы

In [9]:
# Установка формата ячеек
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER',
            "backgroundColor": {
                "red": 0.8,
                "green": 0.8,
                "blue": 0.8,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": True,
               "fontSize": 14
             }
          }
        },
        "range": 
        {
          "sheetId": sheetId,
          "startRowIndex": 1,
          "endRowIndex": 2,
          "startColumnIndex": 1,
          "endColumnIndex": 4
        },
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()

<img src="img/Task09_26.jpg">

Есть простой способ узнать, какую ширину или цвет нужно задать ячейке.
Для этого достаточно вручную отформатировать одну из ячеек и прочитать ее свойства.

In [10]:
ranges = ["Лист номер один!C2:C2"] # 
          
results = service.spreadsheets().get(spreadsheetId = spreadsheetId, 
                                     ranges = ranges, includeGridData = True).execute()
print('Основные данные')
print(results['properties'])
print('\nЗначения и раскраска')
print(results['sheets'][0]['data'][0]['rowData'] )
print('\nВысота ячейки')
print(results['sheets'][0]['data'][0]['rowMetadata'])
print('\nШирина ячейки')
print(results['sheets'][0]['data'][0]['columnMetadata'])


Основные данные
{'title': 'Первый тестовый документ', 'locale': 'ru_RU', 'autoRecalc': 'ON_CHANGE', 'timeZone': 'Etc/GMT', 'defaultFormat': {'backgroundColor': {'red': 1, 'green': 1, 'blue': 1}, 'padding': {'top': 2, 'right': 3, 'bottom': 2, 'left': 3}, 'verticalAlignment': 'BOTTOM', 'wrapStrategy': 'OVERFLOW_CELL', 'textFormat': {'foregroundColor': {}, 'fontFamily': 'arial,sans,sans-serif', 'fontSize': 10, 'bold': False, 'italic': False, 'strikethrough': False, 'underline': False}}}

Значения и раскраска
[{'values': [{'userEnteredValue': {'stringValue': 'Ячейка C2'}, 'effectiveValue': {'stringValue': 'Ячейка C2'}, 'formattedValue': 'Ячейка C2', 'userEnteredFormat': {'backgroundColor': {'red': 0.06666667, 'green': 0.33333334, 'blue': 0.8}, 'horizontalAlignment': 'CENTER', 'textFormat': {'foregroundColor': {'green': 1}, 'fontSize': 14, 'bold': True}}, 'effectiveFormat': {'backgroundColor': {'red': 0.06666667, 'green': 0.33333334, 'blue': 0.8}, 'padding': {'top': 2, 'right': 3, 'bottom':

Этот код выведет свойства ячейки C2. Можно выбрать шрифт и цвет заливки вручную (в таблице), в потом увидеть, как они отражаются в json.

##### Задание для самостоятельного выполнения:

- Сделайте заголовок таблицы цветным, курсивным и крупным. На ваш вкус.

## 6. Чтение данных из таблицы

Чтобы особенности чтения данных проявились в полной мере, я заполнил ячейки B4, C7 и D5 как показано на рисунке.

<img src="img/Task09_27.jpg">

Код для чтения данных

In [16]:
ranges = ["Лист номер один!A2:F8"] # 
          
results = service.spreadsheets().values().batchGet(spreadsheetId = spreadsheetId, 
                                     ranges = ranges, 
                                     valueRenderOption = 'FORMULA',  
                                     dateTimeRenderOption = 'FORMATTED_STRING').execute() 
sheet_values = results['valueRanges'][0]['values']
sheet_values


[['', 'Ячейка B2', 'Ячейка C2', 'Ячейка D2'],
 ['', 25, '=6*6', '=sin(3,14/2)'],
 [],
 ['', '', 'ййй', 1],
 ['', '', 'рррллол', 2],
 ['', '', '', 3]]

Некоторые параметры функции:
valueRenderOption - формат чтения числовых данных.
- FORMATTED_VALUE  - чтение с учетом формата отображения. Т.е. что было видно в таблице, то и прочитается. Например, в ячейке D3 число 0,9999999, но выбран формат “два знака после запятой”, поэтому отображается “1,00”, именно в таком формате оно и прочитается.
- UNFORMATTED_VALUE - читается содержимое ячейки, без учета настроек форматирование (т.е. прочиталось бы 0,9999999)
- FORMULA - отображается формула (в этом случае “=sin(3,14/2)”. Если в ячейке введено число, то в этом режиме оно и прочитается.

Этот код читает данные и построчно выводит их на экран.
Читаемый диапазон A2:F8.

Как видно на экране:
1. Если ни одна ячейка в читаемой строке не заполнена - данные по строке не выводятся.
2. Данные после последней заполненной ячейки не выводятся.


##### Задание для самостоятельного выполнения:

- Создайте и заполните данными новый файл.
- Выдайте доступ своему сервисному аккаунту
- Скопируйте идентификатор файле (в URL-е страницы с открытой таблицей он находится между “https://docs.google.com/spreadsheets/d/” и “/edit#gid=0”).
- Используя этот spreadsheetId прочитайте данные из своего файла и выведите их на экран.
